<a href="https://colab.research.google.com/github/sunc-dev/spaCY-ner-sustain/blob/main/web_text_sustain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install justext
!pip install spacy
!pip install pdfminer.six
!pip install nltk 
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [57]:
import os
import pandas as pd
from pathlib import Path
import glob
from io import StringIO

#pdf miner 
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.high_level import extract_text

#SpaCy 
import en_core_web_md
import spacy
from spacy import displacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.pipeline import Sentencizer
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB
from spacy.lookups import Lookups
#nltk
from nltk import tokenize


In [58]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Scrape request data

In [59]:
import requests
import justext


urls =[
       #sump pumps
       'https://en.wikipedia.org/wiki/Sump_pump',
       'https://www.triadbasementwaterproofing.com/blog/2016/11/dont-make-these-10-common-mistakes-with-your-sump-pump/',
       'https://www.sumppumpsdirect.com/stories/101-How-To-Pick-The-Perfect-Sump-Pump.html',
       'https://www.bobvila.com/articles/some-advice-about-sump-pumps/',
       'https://www.drainkingplumbers.ca/flood-protection/sump-pump/',
       'https://eng.ichacha.net/zaoju/sump%20pump.html',
       'https://glosbe.com/en/fr/sump%20pump',
       'https://sentencedict.com/sump%20pump.html',
      

       #green roofs
       'https://greenroofs.org/about-green-roofs',
       'https://en.wikipedia.org/wiki/Green_roof',
       'https://science.howstuffworks.com/environmental/green-science/green-rooftop.htm',
       'https://www.toronto.ca/city-government/planning-development/official-plan-guidelines/green-roofs/green-roof-overview/',
       'https://www.forbes.com/sites/houzz/2013/12/30/6-busted-green-roof-myths/#3ccc80673384',
       'https://sustainabletechnologies.ca/home/urban-runoff-green-infrastructure/low-impact-development/green-roofs/',
       'https://www.ecohome.net/guides/1099/living-green-roofs-the-secret-to-success/',
       'https://eng.ichacha.net/zaoju/green%20roof.html',
       'https://glosbe.com/en/fr/green%20roof',
       'https://sentencedict.com/wordQueryDo.php',
       'http://www.rhymezone.com/r/rhyme.cgi?Word=sump+pump&typeofrhyme=wke&org1=syl&org2=l&org3=y',

       #back-water valves
       'https://pw.lacounty.gov/general/faq/index.cfm?Action=getAnswers&FaqID=IyNNMzEK&Theme=default&ShowTemplate=',
       'http://www.ibc.ca/on/home/risk-management/mitigation-techniques/backwater-valve',
       'https://www.outofthisworldhomeservices.com/blog/do-i-need-a-backwater-valve/',
       'https://www.economical.com/en/blog/economical-blog/october-2017/backwater-valve-101',
       'http://backwatervalve.com/products/fullport-backwater-valve.html',
       'https://www.citywindsor.ca/residents/environment/climate-change-adaptation/climate-resilient-home/Pages/Backwater-Valve.aspx',
       'https://eng.ichacha.net/zaoju/backwater%20valve.html',
       'https://glosbe.com/en/fr/backwater%20valve',
       'http://www.rhymezone.com/r/rhyme.cgi?Word=green+roof&typeofrhyme=wke&org1=syl&org2=l&org3=y',


       #emergency power system
       'https://en.wikipedia.org/wiki/Emergency_power_system',
       'https://eng.ichacha.net/zaoju/emergency%20standby.html',
       'https://glosbe.com/en/fr/emergency%20power',
       'https://sentencedict.com/wordQueryDo.php',

        #back-up power/generator
       'https://www.generac.com/for-homeowners/home-backup-power',
       'https://trustedpros.ca/articles/electrical/top-five-backup-power-options-for-your-home-electrical-system',
       'https://news.energysage.com/battery-backup-power-vs-generators-which-is-right-for-you/',
       'https://www.nema.org/storm-disaster-recovery/backup-generation/backup-power-systems',
       'https://www.generac.com/all-products/generators/home-backup-generators#?cat=6&cat=214&cat=217&cat=249',
       'https://www.csemag.com/articles/understanding-backup-power-system-fuel-choices/',
       'https://eng.ichacha.net/zaoju/backup%20power.html',
       'https://eng.ichacha.net/zaoju/backup%20generator.html',
       'http://www.rhymezone.com/r/rhyme.cgi?Word=backup+power&typeofrhyme=wke&org1=syl&org2=l&org3=y',

        #standby generator
       'https://www.popularmechanics.com/home/how-to/a8523/should-you-buy-a-standby-generator-14880060/',
       'https://www.iko.com/comm/blog/commercial-roof-drains/',
       'https://eng.ichacha.net/zaoju/standby%20generator.html',
       'https://glosbe.com/en/fr/standby%20generator',


       #roof drain 
       'http://blog.equiparts.net/mifab-roofguards/',
       'https://www.commercialplumbingsupply.com/products.asp?cat=17',
       'https://www.portalsplus.com/roof-drains',
       'https://eng.ichacha.net/zaoju/roof%20drain.html',
       'https://glosbe.com/en/fr/roof%20drain',


       #fuel storage tanks
       'https://www.smallfarmcanada.ca/columns/equipment/fuel-storage-the-pros-and-cons-of-buying-and-storing-fuel-in/',
       'https://www.powermag.com/understanding-diesel-fuel-storage-requirements/',
       'https://www.burnsmcd.com/projects/fuel-storage-tank-replacement',
       'https://eng.ichacha.net/zaoju/fuel%20storage.html',
       'https://glosbe.com/en/fr/fuel%20storage',
       'https://sentencedict.com/wordQueryDo.php',


       #downspouts
       'https://en.wikipedia.org/wiki/Downspout',
       'https://www.designingbuildings.co.uk/wiki/Rainwater_downpipe',
       'https://wikidiff.com/downpipe/downspout',
       'https://www.collinsdictionary.com/dictionary/english-spanish/downspout',
       'https://eng.ichacha.net/zaoju/downspout.html',
       'https://eng.ichacha.net/zaoju/weeping%20tile.html',
       'https://glosbe.com/en/fr/downspout',
       'https://www.translateen.com/sentence/Downspout-in-sentence-examples/',
       'http://www.rhymezone.com/r/rhyme.cgi?loc=ol_ue&typeofrhyme=wke&Word=downspout',


       #weeping tiles
       'https://en.wikipedia.org/wiki/Weeping_tile',
       'https://aquamasterplumbing.com/2015/01/26/everything-you-need-to-know-about-weeping-tile/',
       'https://wikivisually.com/wiki/Weeping_tile',
       'https://glosbe.com/en/fr/weeping%20tile',
  


       #site grading
       'https://nearsay.com/c/362163/338839/what-is-site-grading-how-is-it-used-in-construction#:~:text=Site%20grading%20is%20a%20technique,a%20stronger%2C%20more%20even%20foundation.',
       'https://designeverest.com/blog/what-are-site-grading-plans-and-earthwork-estimates/',
       'https://www.buildingexpertscanada.com/engineering/site_grading',
       'https://www.wisegeek.com/what-is-site-grading.htm',
       'http://www.cowenconstruction.com/a-quick-intro-to-site-grading/',
       'https://glosbe.com/en/fr/site%20grading',
       'https://eng.ichacha.net/zaoju/site%20grading.html',
       'https://www.lawinsider.com/clause/site-grading',
       




       #flood barriers
       'https://en.wikipedia.org/wiki/Flood_barrier',
       'https://www.collinsdictionary.com/dictionary/english/flood-barrier',
       'https://www.thefreedictionary.com/Flood+barrier',
       'https://eng.ichacha.net/zaoju/flood%20barrier.html',
       'https://glosbe.com/en/fr/flood%20barrier',



       #below ground floors
       'https://glosbe.com/en/fr/below%20ground%20floors',
       


       #support rooms
       'https://glosbe.com/en/fr/support%20rooms',
       


       #rain water harvesting systems
       'https://glosbe.com/en/fr/rainwater%20harvesting',
       'https://glosbe.com/en/fr/rainwater%20harvesting%20system',
       'https://eng.ichacha.net/zaoju/rainwater%20harvesting.html',
       'https://www.watercache.com/education/rainwater-harvesting-101',
       'https://www.crd.bc.ca/education/green-stormwater-infrastructure/rainwater-harvesting',
       'https://en.wikipedia.org/wiki/Rainwater_harvesting',


       


       #gutters
      'https://glosbe.com/en/fr/gutters',
      'https://eng.ichacha.net/zaoju/gutter.html',
      'https://en.wikipedia.org/wiki/Rain_gutter',
      'https://yourstudent-gemini.fandom.com/wiki/Rain_gutter',
      'http://www.rhymezone.com/r/rhyme.cgi?Word=gutter&typeofrhyme=wke&org1=syl&org2=l&org3=y'
]



In [60]:
texts = []

for url in urls:
  response = requests.get(url)
  paragraphs = justext.justext(response.content, justext.get_stoplist("English"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      texts.append(paragraph.text)


# Load spacy for model training

In [61]:
#Declare english vocab for our nlp object to define english language sentence boundaries
nlp = spacy.blank("en")
nlp = English()
nlp = en_core_web_md.load()
nlp.max_length = 6130000 # or higher

nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f9d8f3c7358>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f9d91a56228>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f9d91a56288>)]

In [62]:
sentencizer = Sentencizer(punct_chars=[".", "?", "!", "。"])
nlp.add_pipe(sentencizer, before="tagger")
nlp.pipeline

[('sentencizer', <spacy.pipeline.pipes.Sentencizer at 0x7f9da5504828>),
 ('tagger', <spacy.pipeline.pipes.Tagger at 0x7f9d8f3c7358>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f9d91a56228>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f9d91a56288>)]

In [63]:
# alternative code for sentence tokenization using nltk
# tokens = tokenize.sent_tokenize(text)

#sentence tokenization using english vocab and SpaCy Sentencizer
#declare doc object to pass into our nlp pipeline
all_text = " ".join(str(x) for x in texts)    
doc = nlp(all_text)

#check number of sentences detected by Sentencizer
print("Number of lines :", (len(list(doc.sents))))

# create a list of tokenized sentences
sents = [sent.text.strip() for sent in doc.sents]

#lemmatization

#check sents list object
#sents

Number of lines : 2129


In [64]:
### Using Matcher - Need to improve
'''
Define matcher object using english vocab we defined earlier. 
Test using Matcher to define rules based on the token attributes of the string.
'''

LABEL = 'SUSTECH'

token_matcher = Matcher(nlp.vocab, validate=True)

# create token patterns - ADD more token patterns here!
'''
token_patterns = [
    [{"LOWER": "green", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}}, {"LOWER": "roof", 'POS': {'IN' : ['NOUN','PROPN']}}],
    [{"LOWER": "stormwater", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}},{"LOWER": "pond", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}}],
    [{"LOWER": "stormwater", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}},{"LOWER": "ponds", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}}],
    [{"LOWER": "sump", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}},{"LOWER": "pump", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}}],
    [{"LOWER": "sump", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}},{"LOWER": "pumps", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}}],
    [{"LOWER": "backup", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}},{"LOWER": "generator", 'POS': {'IN' : ['ADJ','NOUN', 'PROPN']}}],
]
'''


token_patterns = [
    [{"LOWER": "green"}, {"LOWER": "roof"}],
    [{"LOWER": "sump"}, {"LOWER": "pump"}],
    [{"LOWER": "backwater"},{"LOWER": "valve"}],
    [{"LOWER": "backup"},{"LOWER": "power"}],
    [{"LOWER": "standby"},{"LOWER": "generator"}],
    [{"LOWER": "backup"},{"LOWER": "generator"}],
    [{"LOWER": "emergency"},{"LOWER": "power"}],
    [{"LOWER": "roof"},{"LOWER": "drain"}, {"LOWER": "cover"}],
    [{"LOWER": "roof"},{"LOWER": "drain"}],
    [{"LOWER": "fuel"},{"LOWER": "storage"}, {"LOWER": "tank"}],
    [{"LOWER": "fuel"},{"LOWER": "storage"}],
    [{"LOWER": "downspout"}],
    [{"LOWER": "weeping"},{"LOWER": "tiles"}],
    [{"LOWER": "site"},{"LOWER": "grading"}],
    [{"LOWER": "flood"},{"LOWER": "barrier"}],
    [{"LOWER": "below"},{"LOWER": "ground"}, {"LOWER": "floor"}],
    [{"LOWER": "support"},{"LOWER": "ground"}, {"LOWER": "room"}],
    [{"LOWER": "rainwater"}, {"LOWER": "harvest", "LEMMA": "harvest", "POS": {"IN": ["NOUN","VERB"]}}],
    [{"LOWER": "rainwater"}, {"LOWER": "harvesting", "POS": {"IN": ["NOUN"]}}],
    [{"LOWER": "gutter"}],
    [{"LOWER": "rainwater"}, {"LOWER": "gutter"}],

    #plural patterns
    [{"LOWER": "green"}, {"LOWER": "roofs"}],
    [{"LOWER": "sump"}, {"LOWER": "pumps"}],
    [{"LOWER": "backwater"},{"LOWER": "valves"}],
    [{"LOWER": "backup"},{"LOWER": "powers"}],
    [{"LOWER": "backup"},{"LOWER": "generators"}],
    [{"LOWER": "emergency"},{"LOWER": "powers"}],
    [{"LOWER": "roof"},{"LOWER": "drain"}, {"LOWER": "covers"}],
    [{"LOWER": "roof"},{"LOWER": "drains"}],
    [{"LOWER": "fuel"},{"LOWER": "storage"}, {"LOWER": "tanks"}],
    [{"LOWER": "fuel"},{"LOWER": "storages"}],
    [{"LOWER": "downspouts"}],
    [{"LOWER": "weeping"},{"LOWER": "tiles"}],
    [{"LOWER": "site"},{"LOWER": "gradings"}],
    [{"LOWER": "flood"},{"LOWER": "barriers"}],
    [{"LOWER": "below"},{"LOWER": "ground"}, {"LOWER": "floors"}],
    [{"LOWER": "support"},{"LOWER": "ground"}, {"LOWER": "rooms"}],
    [{"LOWER": "rainwater"}, {"LOWER": "harvests", "LEMMA": "harvests", "POS": {"IN": ["NOUN", "VERB"]}}],
    [{"LOWER": "rainwater"}, {"LOWER": "harvestings", "LEMMA": "harvestings", "POS": {"IN": ["NOUN"]}}],
    [{"LOWER": "gutters"}],
    [{"LOWER": "rainwater"}, {"LOWER": "gutters"}],

]



token_matcher.add(LABEL, None, *token_patterns)


In [65]:
text = (
    '''I have a broken green roof in my Green Roof hood. There is no Stormwater Pond on this roof. I like stormwater ponds. \n
    The backup generator located on the 8th floor is not working. The Sump Pump is so broken. \n
    The rainwater harvest is not working.
    ''')
test_doc = nlp(text)
words_lemmas_list = [token.lemma_ for token in test_doc]

print(test_doc)
print(words_lemmas_list)
#tokenizer = nlp.Defaults.create_tokenizer(nlp)

for idx, start, end in token_matcher(test_doc):
    print(test_doc[start:end],)
    
tokens = [token.text for token in test_doc]
tags = [token.tag for token in test_doc]
displacy.render(test_doc, style='dep', jupyter=True)

I have a broken green roof in my Green Roof hood. There is no Stormwater Pond on this roof. I like stormwater ponds. 

    The backup generator located on the 8th floor is not working. The Sump Pump is so broken. 

    The rainwater harvest is not working.
    
['-PRON-', 'have', 'a', 'break', 'green', 'roof', 'in', '-PRON-', 'Green', 'Roof', 'hood', '.', 'there', 'be', 'no', 'Stormwater', 'Pond', 'on', 'this', 'roof', '.', '-PRON-', 'like', 'stormwater', 'pond', '.', '\n\n    ', 'the', 'backup', 'generator', 'locate', 'on', 'the', '8th', 'floor', 'be', 'not', 'work', '.', 'the', 'Sump', 'Pump', 'be', 'so', 'broken', '.', '\n\n    ', 'the', 'rainwater', 'harvest', 'be', 'not', 'work', '.', '\n    ']
green roof
Green Roof
backup generator
Sump Pump
rainwater harvest


In [66]:
#define our utility function
def train_parser(doc):
    position = [(doc[start:end].start_char, doc[start:end].end_char, LABEL) for 
                  idx, start, end in token_matcher(doc)]
    return (doc.text,  {'entities': position})

In [67]:
train_parser(test_doc)

('I have a broken green roof in my Green Roof hood. There is no Stormwater Pond on this roof. I like stormwater ponds. \n\n    The backup generator located on the 8th floor is not working. The Sump Pump is so broken. \n\n    The rainwater harvest is not working.\n    ',
 {'entities': [(16, 26, 'SUSTECH'),
   (33, 43, 'SUSTECH'),
   (127, 143, 'SUSTECH'),
   (189, 198, 'SUSTECH'),
   (223, 240, 'SUSTECH')]})

In [68]:
TRAIN_DATA = [train_parser(d) for d in nlp.pipe(sents) if len(token_matcher(d))==1]
TRAIN_DATA[0:10]
#len(TRAIN_DATA)

[('A sump pump is a pump used to remove water that has accumulated in a water-collecting sump basin, commonly found in the basements of homes.',
  {'entities': [(2, 11, 'SUSTECH')]}),
 ('Sump pumps are used where basement flooding happens regularly and to solve dampness where the water table is above the foundation of a home.',
  {'entities': [(0, 10, 'SUSTECH')]}),
 ('Sump pumps send water away from a house to any place where it is no longer problematic, such as a municipal storm drain or a dry well.',
  {'entities': [(0, 10, 'SUSTECH')]}),
 ('Municipalities urge homeowners to disconnect and reroute sump pump discharge away from sanitary sewers.',
  {'entities': [(57, 66, 'SUSTECH')]}),
 ('Many homeowners have inherited their sump pump configurations and do not realize that the pump discharges into the sewer.',
  {'entities': [(37, 46, 'SUSTECH')]}),
 ("Usually hardwired into a home's electrical system, sump pumps may have a battery backup.",
  {'entities': [(51, 61, 'SUSTECH')]}),
 (

In [69]:
train_path = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train'
TRAIN_DATA_OUTPUT = pd.DataFrame(TRAIN_DATA, columns=['text',
                                                      'position'])
TRAIN_DATA_OUTPUT.to_csv(os.path.join(train_path,'trainset3.csv'), index=False)